In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr


In [13]:
polling_df = pd.read_csv('../election_resources/cleaned_2019_data.csv')
polling_df.head()

,question_id,end_date,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,114577,2019-12-10,YouGov,1195,rv,Michael F. Bennet,14.0,29.0,4.0,10.0,12.0,17.0,0
1,114585,2019-12-10,YouGov,1198,rv,Michael Bloomberg,24.0,58.0,7.0,17.0,20.0,38.0,0
2,114589,2019-12-10,YouGov,1198,rv,Cory A. Booker,35.0,45.0,11.0,24.0,12.0,33.0,0
3,114593,2019-12-10,YouGov,1198,rv,Steve Bullock,14.0,27.0,3.0,11.0,11.0,16.0,0
4,114597,2019-12-10,YouGov,1202,rv,Pete Buttigieg,35.0,44.0,14.0,21.0,13.0,31.0,0


In [14]:
polling_df.drop(columns=['question_id'], inplace=True)
polling_df.head()

,end_date,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,2019-12-10,YouGov,1195,rv,Michael F. Bennet,14.0,29.0,4.0,10.0,12.0,17.0,0
1,2019-12-10,YouGov,1198,rv,Michael Bloomberg,24.0,58.0,7.0,17.0,20.0,38.0,0
2,2019-12-10,YouGov,1198,rv,Cory A. Booker,35.0,45.0,11.0,24.0,12.0,33.0,0
3,2019-12-10,YouGov,1198,rv,Steve Bullock,14.0,27.0,3.0,11.0,11.0,16.0,0
4,2019-12-10,YouGov,1202,rv,Pete Buttigieg,35.0,44.0,14.0,21.0,13.0,31.0,0


In [15]:
polling_df.nunique()

end_date                133
pollster                 13
sample_size             288
population                1
politician               34
favorable                68
unfavorable              65
very_favorable           49
somewhat_favorable       43
somewhat_unfavorable     33
very_unfavorable         60
results                   2
dtype: int64

In [16]:
polling_df['pollster'].value_counts()

YouGov                          949
HarrisX                         271
Morning Consult                 195
Harris Poll                      68
Quinnipiac University            48
CNN/SSRS                         46
Monmouth University              19
Fox News                         17
ABC News/Washington Post          6
Marist College                    5
Suffolk University                4
Public Policy Polling             2
NBC News/Wall Street Journal      1
Name: pollster, dtype: int64

In [17]:
pollster_to_replace = list(polling_df['pollster'].value_counts()[polling_df['pollster'].value_counts() < 10].index)
for pollster in pollster_to_replace:
    polling_df['pollster'] = polling_df['pollster'].replace(pollster, 'pollster<10')
polling_df['pollster'].value_counts()

YouGov                   949
HarrisX                  271
Morning Consult          195
Harris Poll               68
Quinnipiac University     48
CNN/SSRS                  46
Monmouth University       19
pollster<10               18
Fox News                  17
Name: pollster, dtype: int64

In [18]:
polling_df['politician'].value_counts()

Donald Trump             135
Elizabeth Warren         101
Bernard Sanders           99
Joseph R. Biden Jr.       95
Kamala D. Harris          91
Pete Buttigieg            88
Cory A. Booker            77
Amy Klobuchar             75
Julián Castro             72
Beto O'Rourke             71
Tulsi Gabbard             66
Andrew Yang               63
John K. Delaney           62
Kirsten E. Gillibrand     50
Marianne Williamson       46
Tim Ryan                  41
Michael F. Bennet         41
Wayne Messam              40
Steve Bullock             40
Jay Robert Inslee         40
John Hickenlooper         39
Tom Steyer                32
Bill de Blasio            30
Joe Sestak                29
Seth Moulton              26
Eric Swalwell             22
Mike Gravel               21
Michael Bloomberg         12
Sherrod Brown              8
Stacey Yvonne Abrams       7
Deval Patrick              5
Terry R. McAuliffe         3
Eric H. Holder             2
Eric Garcetti              2
Name: politici

In [19]:
politician_to_replace = list(polling_df['politician'].value_counts()[polling_df['politician'].value_counts() < 10].index)
for i in politician_to_replace:
    polling_df['politician'] = polling_df['politician'].replace(i, 'politician<10')
polling_df['politician'].value_counts()

Donald Trump             135
Elizabeth Warren         101
Bernard Sanders           99
Joseph R. Biden Jr.       95
Kamala D. Harris          91
Pete Buttigieg            88
Cory A. Booker            77
Amy Klobuchar             75
Julián Castro             72
Beto O'Rourke             71
Tulsi Gabbard             66
Andrew Yang               63
John K. Delaney           62
Kirsten E. Gillibrand     50
Marianne Williamson       46
Tim Ryan                  41
Michael F. Bennet         41
Wayne Messam              40
Steve Bullock             40
Jay Robert Inslee         40
John Hickenlooper         39
Tom Steyer                32
Bill de Blasio            30
Joe Sestak                29
politician<10             27
Seth Moulton              26
Eric Swalwell             22
Mike Gravel               21
Michael Bloomberg         12
Name: politician, dtype: int64

In [20]:
polling_df = pd.get_dummies(polling_df, dtype=float)
polling_df.head()

,sample_size,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results,end_date_2019-01-04,end_date_2019-01-06,...,politician_Michael F. Bennet,politician_Mike Gravel,politician_Pete Buttigieg,politician_Seth Moulton,politician_Steve Bullock,politician_Tim Ryan,politician_Tom Steyer,politician_Tulsi Gabbard,politician_Wayne Messam,politician_politician<10
0,1195,14.0,29.0,4.0,10.0,12.0,17.0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1198,24.0,58.0,7.0,17.0,20.0,38.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1198,35.0,45.0,11.0,24.0,12.0,33.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1198,14.0,27.0,3.0,11.0,11.0,16.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1202,35.0,44.0,14.0,21.0,13.0,31.0,0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
y = polling_df['results'].values
X = polling_df.drop(['results'], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

In [22]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'leaky_relu', 'elu', 'selu'])
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
                                        min_value=1,
                                        max_value=100,
                                        step=2), activation=activation, input_dim=len(X_train_scaled[0])))
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                        min_value=1,
                                        max_value=100,
                                        step=2), activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    loss = hp.Choice('loss', ['binary_crossentropy', 'mse', 'mae', 'mape', 'msle', 'kld', 'cosine_proximity'])
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd', 'adagrad', 'adadelta', 'adamax', 'nadam'])
    nn_model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    
    return nn_model

In [28]:
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=20, hyperband_iterations=10, directory='my_dir2', project_name='intro_to_kt')

In [29]:
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 263 Complete [00h 00m 09s]
val_accuracy: 0.8529411554336548

Best val_accuracy So Far: 0.8529411554336548
Total elapsed time: 00h 14m 33s
INFO:tensorflow:Oracle triggered exit


In [31]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'selu',
 'first_units': 85,
 'num_layers': 9,
 'units_0': 9,
 'loss': 'mse',
 'optimizer': 'rmsprop',
 'units_1': 65,
 'units_2': 19,
 'units_3': 77,
 'units_4': 9,
 'units_5': 91,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0,
 'units_6': 1,
 'units_7': 1,
 'units_8': 1}

In [30]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: nan - accuracy: 0.8529 - 267ms/epoch - 21ms/step
Loss: nan, Accuracy: 0.8529411554336548


In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  85
hidden_nodes_layer2 = 9
hidden_nodes_layer3 = 65
hidden_nodes_layer4 = 19
hidden_nodes_layer5 = 77
hidden_nodes_layer6 = 9
hidden_nodes_layer7 = 91
hidden_nodes_layer8 = 1
hidden_nodes_layer9 = 1

output_dim = 1



nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="selu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer7, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer8, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer9, activation="selu"))




# Output layer
nn_model.add(tf.keras.layers.Dense(output_dim, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 85)                15300     
                                                                 
 dense_12 (Dense)            (None, 9)                 774       
                                                                 
 dense_13 (Dense)            (None, 65)                650       
                                                                 
 dense_14 (Dense)            (None, 19)                1254      
                                                                 
 dense_15 (Dense)            (None, 77)                1540      
                                                                 
 dense_16 (Dense)            (None, 9)                 702       
                                                                 
 dense_17 (Dense)            (None, 91)               

In [37]:
nn_model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [38]:

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, initial_epoch= 0)

Epoch 1/100
39/39 [==============================] - 1s 8ms/step - loss: nan - accuracy: 0.8610
Epoch 2/100
39/39 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.8610
Epoch 3/100
39/39 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8610
Epoch 4/100
39/39 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8610
Epoch 5/100
39/39 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8610
Epoch 6/100
39/39 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8610
Epoch 7/100
39/39 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8610
Epoch 8/100
39/39 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8610
Epoch 9/100
39/39 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8610
Epoch 10/100
39/39 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8610
Epoch 11/100
39/39 [===================

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: nan - accuracy: 0.8529 - 143ms/epoch - 11ms/step
Loss: nan, Accuracy: 0.8529411554336548


In [36]:
print(nn_model.predict(X_test_scaled))

13/13 [==============================] - 3s 3ms/step
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [